In [56]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import os
from bs4 import BeautifulSoup
import json
import re


In [57]:
# instantiate a chrome options object so you can set the size and headless preference
chrome_options = Options()
# chrome_options.add_argument("--headless")
chrome_options.add_argument("--window-size=1920x1080")

driver = webdriver.Chrome(chrome_options=chrome_options)

/Users/Pengji/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  


In [8]:
for num in range(1,231):
    driver.get('https://www.cryst.ehu.es/cryst/get_wp.html')

    inputElement = driver.find_element_by_name("gnum")
    inputElement.send_keys(num)
    driver.find_element_by_name('standard').click()

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    table = soup.find('center')
    table = table.find('tbody')
    
    Wyckoff_positions = {}
    i = 0
    for row in table.find_all('tr',recursive=False)[2:]:
        Wyckoff_site = row.find_all('td')[1].text
        position = row.find_all('td')[3].text.split(' ')[0].strip("()").split(',')
        Wyckoff_positions[Wyckoff_site] = position
        i+=1
    with open(f'space_group_{num}_Wyckoff_site_data.json', 'w') as f:
        json.dump(Wyckoff_positions, f)


/Users/Pengji/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  


In [ ]:
# use Rhombohedral axes choice for hR structures

letter = 'abcdefghijklmnopqrstuvwxyz'

for num in (146, 148, 155, 160, 161, 166, 167):

    Wyckoff_positions_list = []
    Wyckoff_positions = {}

    driver.get(f'https://it.iucr.org/Ac/ch2o3v0001/sgtable2o3o{num}/')
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    table = soup.find_all('table', {'class': 'genpos'})[-1]
    table = table.find('table', {'class': 'genposcoords'})
    genpos = table.find('td', {'class': 'genposcoords'})
    genpos = re.findall(r"<i><i>(.*?)</i></i>", str(genpos))
    Wyckoff_positions_list.append(genpos)

    table = soup.find_all('table', {'class': 'specpos'})[-1]
    table = table.find_all('table', {'class': 'specposcoords'})
    for line in table:
        pos = pd.read_html(str(line))[0].iloc[0, 0].replace(u'\xa0', u'').split(',')
        Wyckoff_positions_list.append(pos)
    n = len(Wyckoff_positions_list)
    for i in range(0, n):
        Wyckoff_positions[letter[n - 1 - i]] = Wyckoff_positions_list[i]

    with open('space_group_{}_Wyckoff_site_data.json'.format(num), 'w') as f:
        json.dump(Wyckoff_positions, f)
